In [1]:
# !pip install deeptables[gpu]

In [2]:
from deeptables.models.deeptable import DeepTable, ModelConfig
from deeptables.models.deepnets import WideDeep
from deeptables.datasets import dsutils
from sklearn.model_selection import train_test_split



In [3]:
import numpy as np
import pandas as pd
import os
%matplotlib inline
import matplotlib.pyplot as plt 
import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')
from tqdm import tqdm
tqdm.pandas()
from catboost import CatBoostClassifier

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, cross_val_score, RepeatedStratifiedKFold
from sklearn import metrics
from lightgbm import LGBMClassifier
from IPython.display import FileLink

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [5]:
train_file = '/content/gdrive/My Drive/train.csv'
test_file = '/content/gdrive/My Drive/test.csv'
filename = 'result.csv'

In [6]:
df, tdf = pd.read_csv(train_file), pd.read_csv(test_file)

In [7]:
df.shape, tdf.shape, df.id.nunique(), tdf.id.nunique()

((381109, 12), (127037, 11), 381109, 127037)

In [8]:
cat_cols = ['Gender', 'Driving_License', 'Region_Code',
        'Previously_Insured', 'Vehicle_Age', 'Vehicle_Damage']
reg_cols = [ 'Age', 'Annual_Premium','Vintage']
useless_cols = ['id']
target_col = ['Response']
spl_cat_col = ['Policy_Sales_Channel']

In [9]:
X = df[df.columns[~(df.columns.isin(['id', target_col[0]]))].tolist()]

y = df[target_col[0]].values
print(type(X), type(y))

<class 'pandas.core.frame.DataFrame'> <class 'numpy.ndarray'>


In [10]:
conf = ModelConfig(nets=WideDeep, metrics=['AUC'], auto_categorize=True,apply_class_weight=True)# categorical_columns=cat_cols + spl_cat_col,
                  #  auto_discrete=True)
dt = DeepTable(config=conf)

In [11]:
model, history = dt.fit(X, y, epochs=100)

2 class detected, {0, 1}, so inferred as a [binary classification] task
Preparing features cost:1.2656254768371582
Imputation cost:0.9044692516326904
Categorical encoding cost:1.4498977661132812
fit_transform cost:3.7261109352111816
Calc classes weight.
Examples:
Total:381109
class 0:0.5698417160338398
class 1:4.079522586169985
Injected a callback [EarlyStopping]. monitor:val_AUC, patience:1, mode:max
1 Physical GPUs, 1 Logical GPUs
>>>>>>>>>>>>>>>>>>>>>> Model Desc <<<<<<<<<<<<<<<<<<<<<<< 
---------------------------------------------------------
inputs:
---------------------------------------------------------
['all_categorical_vars: (9)', 'input_continuous_all: (7)']
---------------------------------------------------------
embeddings:
---------------------------------------------------------
input_dims: [4, 5, 4, 68, 4, 55, 4, 157, 292]
output_dims: [4, 4, 4, 4, 4, 4, 4, 4, 4]
dropout: 0.3
---------------------------------------------------------
dense: dropout: 0
batch_normalizati

In [12]:
X_test = tdf[tdf.columns[~(tdf.columns.isin(['id', target_col[0]]))].tolist()]

In [13]:
X.shape, X_test.shape

((381109, 10), (127037, 10))

In [14]:
preds = dt.predict(X_test)

transform_X cost:11.469396829605103
predict_proba cost:13.105039119720459


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:289: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [15]:
sum([1 for i in y if i == 1]) / len(y), len(y)

(0.12256336113815208, 381109)

In [16]:
sum([1 for i in preds if i == 1]) / len(preds), len(preds)

(0.40761352991648103, 127037)

In [17]:
result = pd.DataFrame({'id': tdf['id'], 'Response': preds.ravel()})

In [18]:
result.to_csv(filename, index=False)
FileLink(filename)

/content/result.csv

In [19]:
from google.colab import files
files.download(filename)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>